In [154]:
import warnings
warnings.filterwarnings("ignore")


# Data Manipulation
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk
import itertools
from scipy import stats

# Machine Learning
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import tree

import xgboost as xgb
# from mlxtend.classifier import StackingClassifier

# Ensembling
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier

# Metrics
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix

# Loading data
train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")
frames = [train, test]

In [155]:
for i in frames:
    i.Age = pd.cut(i.Age.fillna(-0.5), (-1, 0, 5, 15, 25, 40, 60,120),
                        labels=["Null", "baby","child", "teen", "adult", "senior", "very_senior"])

    i.Fare = pd.cut(i.Fare.fillna(1), (-1,10,15,20,25,30,50,1000),labels=[1,2,3,4,5,6,7])
    
    # Introducing new features 1. Alone ?? 2. Family Size from (Parch and SibSp) 
    # as these two features are highly corelated
    i['FamilySize'] = i['SibSp'] + i['Parch'] + 1
    i['IsAlone'] = 0
    i.loc[i['FamilySize'] == 1, 'IsAlone'] = 1

In [156]:
for i in frames:
    i["Age"] = preprocessing.LabelEncoder().fit(i["Age"]).transform(i["Age"])
    i["Fare"] = preprocessing.LabelEncoder().fit(i["Fare"]).transform(i["Fare"])
    i["Sex"] = preprocessing.LabelEncoder().fit(i["Sex"]).transform(i["Sex"])

    i.Embarked = i.Embarked.fillna("S")
    i["Embarked"] = preprocessing.LabelEncoder().fit(i["Embarked"]).transform(i["Embarked"])


In [246]:
X_train, X_test, y_train, y_test = train_test_split(train[["Sex", "Age", "Pclass", "Embarked", "Fare", "IsAlone", "FamilySize"]],
                                                        train["Survived"], test_size=.3)
result = pd.DataFrame(columns = ("Classifiers","Training","Testing"))
classifier, train_scores, test_scores = [],[],[]

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, y_train)
classifier.append("RandomForestClassifier")
train_scores.append(random_forest.score(X_train, y_train))
test_scores.append(random_forest.score(X_test, y_test))

cf = SVC()
cf.fit(X_train, y_train)
classifier.append("SVM")
train_scores.append(cf.score(X_train, y_train))
test_scores.append(cf.score(X_test, y_test))

knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, y_train)
classifier.append("KNeighborsClassifier")
train_scores.append(knn.score(X_train, y_train))
test_scores.append(knn.score(X_test, y_test))

clf_log = LogisticRegression()
clf_log = clf_log.fit(X_train,y_train)
classifier.append("LogisticRegression")
train_scores.append(clf_log.score(X_train, y_train))
test_scores.append(clf_log.score(X_test, y_test))

# pd.DataFrame(list(zip(X_train.columns, np.transpose(clf_log.coef_))))

clf_pctr = Perceptron(
    class_weight='balanced'
    )
clf_pctr = clf_pctr.fit(X_train,y_train)
classifier.append("Perceptron")
train_scores.append(clf_pctr.score(X_train, y_train))
test_scores.append(clf_pctr.score(X_test, y_test))


clf_tree = tree.DecisionTreeClassifier(
    #max_depth=3,\
    class_weight="balanced",\
    min_weight_fraction_leaf=0.01\
    )
clf_tree = clf_tree.fit(X_train,y_train)
classifier.append("DecisionTreeClassifier")
train_scores.append(clf_tree.score(X_train, y_train))
test_scores.append(clf_tree.score(X_test, y_test))


clf_ext = ExtraTreesClassifier(
    max_features='auto',
    bootstrap=True,
    oob_score=True,
    n_estimators=1000,
    max_depth=None,
    min_samples_split=10
    #class_weight="balanced",
    #min_weight_fraction_leaf=0.02
    )
clf_ext = clf_ext.fit(X_train,y_train)
classifier.append("ExtraTreesClassifier")
train_scores.append(clf_ext.score(X_train, y_train))
test_scores.append(clf_ext.score(X_test, y_test))


clf_gb = GradientBoostingClassifier(
            #loss='exponential',
            n_estimators=1000,
            learning_rate=0.1,
            max_depth=3,
            subsample=0.5,
            random_state=0).fit(X_train,y_train)
classifier.append("GradientBoostingClassifier")
train_scores.append(clf_gb.score(X_train, y_train))
test_scores.append(clf_gb.score(X_test, y_test))


clf_ada = AdaBoostClassifier(n_estimators=400, learning_rate=0.1)
clf_ada.fit(X_train,y_train)
classifier.append("AdaBoostClassifier")
train_scores.append(clf_ada.score(X_train, y_train))
test_scores.append(clf_ada.score(X_test, y_test))


clf_xgb = xgb.XGBClassifier(
    max_depth=2,
    n_estimators=500,
    subsample=0.5,
    learning_rate=0.1
    )
clf_xgb.fit(X_train,y_train)
classifier.append("XGBClassifier")
train_scores.append(clf_xgb.score(X_train, y_train))
test_scores.append(clf_xgb.score(X_test, y_test))

for i,text in enumerate(classifier):
    result.loc[i+1] = [text,np.around(train_scores[i]*100, decimals = 1),np.around(test_scores[i]*100, decimals=1)]
result.sort("Testing", ascending = False)

,Classifiers,Training,Testing
10,XGBClassifier,85.9,82.1
1,RandomForestClassifier,89.4,81.3
8,GradientBoostingClassifier,88.9,81.3
7,ExtraTreesClassifier,86.0,81.0
9,AdaBoostClassifier,81.9,81.0
2,SVM,83.5,80.6
6,DecisionTreeClassifier,85.1,80.2
4,LogisticRegression,78.8,79.1
3,KNeighborsClassifier,85.7,78.4
5,Perceptron,51.8,50.7


In [239]:
l = []
for i in clf_log.coef_[0]:
    l.append(i)
r = []
for i in clf_pctr.coef_[0]:
    r.append(i)


summary = pd.DataFrame(list(zip(X_train.columns, \
    np.transpose(random_forest.feature_importances_), \
    np.transpose(l), \
#     np.transpose(r), \
    np.transpose(clf_ext.feature_importances_), \
    np.transpose(clf_gb.feature_importances_), \
    np.transpose(clf_ada.feature_importances_), \
    np.transpose(clf_xgb.feature_importances_), \
    )), columns=['Feature','RF','LR','ExtraTree','GB','Ada','Xtreme'])

summary['Median'] = summary.median(1)
summary.sort('Median', ascending=False)


,Feature,RF,LR,ExtraTree,GB,Ada,Xtreme,Median
1,Age,0.193236,-0.036160,0.095927,0.278537,0.3850,0.277288,0.235262
4,Fare,0.178939,0.269263,0.120442,0.235579,0.0850,0.218305,0.198622
6,FamilySize,0.105195,-0.348106,0.066045,0.135932,0.3225,0.123390,0.114293
0,Sex,0.347987,-2.536786,0.471357,0.113535,0.1150,0.108475,0.114267
2,Pclass,0.099075,-0.245789,0.149564,0.099452,0.0425,0.101017,0.099263
3,Embarked,0.055889,-0.104297,0.052945,0.108999,0.0375,0.105085,0.054417
5,IsAlone,0.019678,-0.343450,0.043720,0.027967,0.0125,0.066441,0.023822


In [250]:
clf_vote = VotingClassifier(
    estimators=[
        #('tree', clf_tree),
        ('knn', knn),
        ('svm', cf),
        ('extra', clf_ext),
       #('gb', clf_gb),
        ('xgb', clf_xgb),
        ('percep', clf_pctr),
        ('logistic', clf_log),
        #('RF', clf_rf),
        ],
    weights=[2,2,3,3,1,2],
    voting='hard')
clf_vote.fit(X_train,y_train)
print(clf_vote.score(X_train, y_train))
print(clf_vote.score(X_test, y_test))
scores = cross_val_score(clf_vote, X_train, y_train, cv=5, scoring='accuracy')
print("Voting: Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

0.85393258427
0.817164179104
Voting: Accuracy: 0.80 (+/- 0.02)


In [249]:
abc = pd.DataFrame()
abc["PassengerId"] = test["PassengerId"]
abc["Survived"]=clf_vote.predict(test[["Sex", "Age", "Pclass", "Embarked", "Fare", "IsAlone", "FamilySize"]])
abc.to_csv("test_3061.csv", sep=',' ,index=False )